In [1]:
import pandas as pd #Dataframe, Series
import numpy as np

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

from matplotlib import pyplot as plt
import seaborn as sns

import graphviz
import pydotplus
import io

from scipy import misc

from sklearn.metrics import accuracy_score

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

In [2]:
cid ="cid" 
secret = "secret"
username = "1110989883"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-library-read playlist-read-private'
token = util.prompt_for_user_token(username,scope,client_id='cid',client_secret='secret',redirect_uri='http://localhost:8888/callback/')

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=4624360392134c3c815c37fbe2c380ca&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback%2F&scope=playlist-read-private+user-library-read in your browser


Enter the URL you were redirected to: http://localhost:8888/callback/?code=AQCaE60_qjMoF5uUy2N46TlnznyK2TbW-MFGOe3D-uee6Q0Cft_8syYDiZr61a7fJEYEJ8aENn3ThSX6Exba1rFqOnruio5NNh84AYO790fuJBLJ0VudUiXGFVb_qaioIqqcCB2-LMIwUtWbm0q-clQoL77pG2AOgfTjJm-oVhoT0V8qyHqIOoT92BKqjfg8pTV3HVr5P58KqWLC_wipYYQOlLmOmRJBnRnXVDlCUe_IUPd909iLDeJCQhYcQG4




In [3]:
#key to our analysis of the audio tracks in the sample playlists is the Audio Features GET which shows things like the track's danceability and tempo. I mainly like non-dancey slow tempo music, so this will be especially interesting. See more info here: https://developer.spotify.com/web-api/get-audio-features/.

good_playlist = sp.user_playlist("userID", "playlistID") #user ID, then playlist ID
good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"] 
while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)
good_ids = [] 
for i in range(len(good_songs)- 500):
    good_ids.append(good_songs[i]['track']['id'])
    
#Now we do the same for the bad playlist:

bad_playlist = sp.user_playlist("userID", "playlistID") #user ID, then playlist ID
bad_tracks = bad_playlist["tracks"]
bad_songs = bad_tracks["items"] 
while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)
bad_ids = [] 
for i in range(len(bad_songs)- 500):
    bad_ids.append(bad_songs[i]['track']['id'])

In [4]:
features = []
for i in range(0,len(good_ids), 50):
    audio_features = sp.audio_features(good_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 1
        
for i in range(0,len(bad_ids), 50):
    audio_features = sp.audio_features(bad_ids[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 0
#after iterating through the good and bad playlists and pulling song IDs, now we analuse the audio features of those songs. We are splitting the IDs by 50 because the audio features call can only pull the features for 50 songs at once. After this, we add the features to an array.

In [5]:
trainingData = pd.DataFrame(features)
#graphically representing the songs


pos_tempo = trainingData[trainingData['target'] == 1]['tempo']
neg_tempo = trainingData[trainingData['target'] == 0]['tempo']
pos_dance = trainingData[trainingData['target'] == 1]['danceability']
neg_dance = trainingData[trainingData['target'] == 0]['danceability']
pos_duration = trainingData[trainingData['target'] == 1]['duration_ms']
neg_duration = trainingData[trainingData['target'] == 0]['duration_ms']
pos_loudness = trainingData[trainingData['target'] == 1]['loudness']
neg_loudness = trainingData[trainingData['target'] == 0]['loudness']
pos_speechiness = trainingData[trainingData['target'] == 1]['speechiness']
neg_speechiness = trainingData[trainingData['target'] == 0]['speechiness']
pos_valence = trainingData[trainingData['target'] == 1]['valence']
neg_valence = trainingData[trainingData['target'] == 0]['valence']
pos_energy = trainingData[trainingData['target'] == 1]['energy']
neg_energy = trainingData[trainingData['target'] == 0]['energy']
pos_acousticness = trainingData[trainingData['target'] == 1]['acousticness']
neg_acousticness = trainingData[trainingData['target'] == 0]['acousticness']
pos_key = trainingData[trainingData['target'] == 1]['key']
neg_key = trainingData[trainingData['target'] == 0]['key']
pos_instrumentalness = trainingData[trainingData['target'] == 1]['instrumentalness']
neg_instrumentalness = trainingData[trainingData['target'] == 0]['instrumentalness']

In [6]:
fig = plt.figure(figsize=(12,8))
plt.title("Song Tempo Like / Dislike Distribution")
pos_tempo.hist(alpha=0.7, bins=30, label='positive')
neg_tempo.hist(alpha=0.7, bins=30, label='negative')
plt.legend(loc='upper right')

In [7]:
train, test = train_test_split(trainingData, test_size = 0.15)
print("Training size: {}, Test size:{}".format(len(train),len(test)))

Training size: 2238, Test size:396


In [8]:
#these are the features we need to look at
features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness", "duration_ms"]

#data now in x and y test/train sets and send them into classifiers
x_train = train[features]
y_train = train["target"]

x_test = test[features]
y_test = test["target"]

In [9]:
#using decision tree to check the accuracy of predictions

c = DecisionTreeClassifier(min_samples_split=100)
dt = c.fit(x_train, y_train)
y_pred = c.predict(x_test)
score = accuracy_score(y_test, y_pred) * 100
print("Accuracy using Decision Tree: ", round(score, 1), "%")

Accuracy using Decision Tree:  79.8 %


In [10]:
def show_tree(InputTree, features, path):
    f = io.StringIO()
    tree.export_graphviz(InputTree, out_file=f, feature_names=features)
    pydotplus.graph_from_dot_data(f.getvalue()).write_png(path)
    img = misc.imread(path)
    plt.rcParams["figure.figsize"] = (20,20)
    plt.imshow(img)
show_tree(dt, features, "dec_tree.png")

InvocationException: GraphViz's executables not found

In [ ]:
#using k nearest neighbor to predict enjoyment accuracy 

knn = KNeighborsClassifier(3)
knn.fit(x_train, y_train)
knn_pred = c.predict(x_test)
score = accuracy_score(y_test, knn_pred) * 100
print("Accuracy using Knn Tree: ", round(score, 1), "%")

In [ ]:
#ada starts by creating a learner and then use the results of classification to modify that learner to make it better at predictions. 
#AdaBoost fits that learner then changes the way it predicts between each iteration of data

ada = AdaBoostClassifier(n_estimators=100)
ada.fit(x_train, y_train)
ada_pred = ada.predict(x_test)
score = accuracy_score(y_test, ada_pred) * 100
print("Accuracy using ada: ", round(score, 1), "%")

In [ ]:
#gradient boosting uses the loss function (a measure of how far the prediction was) and tries to reduce it with each iteration

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=0)
gbc.fit(x_train, y_train)
predicted = gbc.predict(x_test)
score = accuracy_score(y_test, predicted)*100
print("Accuracy using Gbc: ", round(score, 1), "%")

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(x_train, y_train)
mlp_pred = mlp.predict(x_test)
score = accuracy_score(y_test, mlp_pred) * 100
print("Accuracy using mlp Tree: ", round(score, 1), "%")

In [ ]:
from sklearn.naive_bayes import GaussianNB
gauss = GaussianNB()
gauss.fit(x_train, y_train)
gauss_pred = gauss.predict(x_test)
score = accuracy_score(y_test, gauss_pred)*100
print("Accuracy using gauss: ", round(score, 1), "%")

In [ ]:
from sklearn.cluster import KMeans
k_means = KMeans(n_clusters=3, random_state=0)
k_means.fit(x_train, y_train)
predicted= k_means.predict(x_test)
score = accuracy_score(y_test, predicted)*100
print("Accuracy using Kmeans: ", round(score, 1), "%")

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_train, y_train)
qda_pred = qda.predict(x_test)
score = accuracy_score(y_test, qda_pred)*100
print("Accuracy using qda: ", round(score, 1), "%")

In [ ]:
from sklearn.svm import SVC
svc_lin = SVC(kernel="linear", C=0.025)
svc_lin.fit(x_train, y_train)
svc_pred = svc_lin.predict(x_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, svc_pred) * 100
print("Accuracy using svc linear: ", round(score, 1), "%")

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
gpc = GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True)
gpc.fit(x_train, y_train)
gpc_pred = gpc.predict(x_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, gpc_pred) * 100
print("Accuracy using gpc: ", round(score, 1), "%")

In [11]:
# Request to modify playlists
username = "userID"
scope = 'playlist-modify-private playlist-modify-public playlist-read-private user-library-read'
token = util.prompt_for_user_token(username, scope)
#Good Playlist
if token:
    sp = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=4624360392134c3c815c37fbe2c380ca&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback%2F&scope=playlist-modify-private+playlist-modify-public+playlist-read-private+user-library-read in your browser


Enter the URL you were redirected to: http://localhost:8888/callback/?code=AQBPIUlJ0QkK3_DYs6BbHIPeLs6kiwl8Fmi8N14bovwLqp2ZCwFgoNDVncxKqgSaLEMLeMbvRqT3US57ULNWhw7ttle58XCB8Hq27SvaHa5fGzKWlkoyDr1q7aYx6GFp685sM--2yr9YUjGz014vqojbdtna9u_quMreR1tuVhihTYgGEjlZop-FjVkVaMEzAt5b0vjlVBhtSV7n2eloxFs1Yjxj2829lms1tRVVVp_usp1WG2TXlJSggCBT0sFBtQ9Qpv21vbJy1lpp4phVhIpICckYZVrbs7HTfmiZA4KphE914aUDs6X6C8o_54oc




In [13]:
playlistToFindSongsYouLikeIn = sp.user_playlist("spotify", "playlistID")

newPlaylist_tracks = playlistToFindSongsYouLikeIn["tracks"]
newPlaylist_songs = newPlaylist_tracks["items"] 
while newPlaylist_tracks['next']:
    newPlaylist_tracks = sp.next(newPlaylist_tracks)
    for song in newPlaylist_tracks["items"]:
        newPlaylist_songs.append(song)
        
newPlaylist_song_ids = [] 
print(len(newPlaylist_songs))
for i in range(len(newPlaylist_songs)):
    newPlaylist_song_ids.append(newPlaylist_songs[i]['track']['id'])
    
newPlaylist_features = []
j = 0
for i in range(0,len(newPlaylist_song_ids),50):
    audio_features = sp.audio_features(newPlaylist_song_ids[i:i+50])
    for track in audio_features:
        track['song_title'] = newPlaylist_songs[j]['track']['name']
        track['artist'] = newPlaylist_songs[j]['track']['artists'][0]['name']
        j= j + 1
        newPlaylist_features.append(track)
print(len(newPlaylist_features))

playlistToLookAtFeatures = pd.DataFrame(newPlaylist_features)

30
30


In [16]:
ada = AdaBoostClassifier(n_estimators=100)
ada.fit(x_train, y_train)
pred = ada.predict(playlistToLookAtFeatures[features])

likedSongs = 0
i = 0
for prediction in pred:
    if(prediction == 1):
        print ("Song: " + playlistToLookAtFeatures["song_title"][i] + ", By: "+ playlistToLookAtFeatures["artist"][i])
        sp.user_playlist_add_tracks("spotify", "playlistID", [test['id'][i]])
        likedSongs= likedSongs + 1
    i = i +1

Song: Doomed, By: Moses Sumney


KeyError: 0

In [20]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=0)
gbc.fit(x_train, y_train)
predicted = gbc.predict(x_test)
score = accuracy_score(y_test, predicted)*100
print("Accuracy using Gbc: ", round(score, 1), "%")

pred = gbc.predict(playlistToLookAtFeatures[features])
i = 0
for prediction in pred:
    if(prediction == 1):
        print ("Song: " + playlistToLookAtFeatures["song_title"][i] + ", By: "+ playlistToLookAtFeatures["artist"][i])
        sp.user_playlist_add_tracks('userID', 'playlistID', playlistToLookAtFeatures["id"])
    i = i +1

Accuracy using Gbc:  80.1 %
Song: Doomed, By: Moses Sumney
Song: The Single Petal Of A Rose - The Queen's Suite, By: Duke Ellington
Song: Wheels Within Wheels, By: Penguin Cafe
Song: Wave Is Not the Water, By: Wye Oak
Song: Bach, By: Dan Reeder
Song: Fight or Flight, By: Bob Reynolds
Song: Familiarity, By: Punch Brothers
Song: Consonance, By: Golden Retriever
Song: Caledonia, My Love, By: Hiss Golden Messenger
Song: La Valse, By: Lester Ash
Song: The Boy In The Gap, By: Martin Hayes Quartet
Song: Don't Think Twice, It's All Right, By: Cicada Rhythm
Song: A Nearer Sun, By: The Westerlies
Song: Elation, By: Aldous Harding
Song: Free Translator, By: The Books
Song: Marginalia #18, By: Masakatsu Takagi
Song: Seeing Things, By: Peter Broderick
Song: Let This Remain, By: Alana Henderson
Song: Let It Be a Dream, By: Silver Torches
Song: How It Feels, By: Ages and Ages
Song: House Of My Soul (You Light The Rooms), By: Langhorne Slim
Song: Bright and Future, By: Henry Jamison
Song: Second Life,